In [1]:
# default_exp metrics

In [2]:
#export
import torch.nn.utils as nn_utils
from fastai.torch_basics import *
from fastai.data.all import *
from fastai.basics import *
from fastai.metrics import *
from dataclasses import field,asdict
from typing import List,Any,Dict,Callable
from collections import deque
import gym
from torch.optim import *

from fastrl.data import *
from fastrl.async_data import *
from fastrl.basic_agents import *
from fastrl.learner import *
from fastai.callback.progress import *

if IN_NOTEBOOK:
    from IPython import display
    import PIL.Image

# RL Metrics

> Metrics for tracking the progress of reinforcement learning agents.

In [37]:
# export
class AvgEpisodeRewardMetric(Metric):
    def __init__(self):self.rolling_rewards=deque([0],maxlen=100)
        
    def accumulate(self,learn):
        yb=learn.yb[0]
#         print(yb)
        yb=[Experience(**{k:yb[k][i] for k in yb}) for i in range(yb['absolute_end'].shape[0])]
#         yb=[for yb.items()]
        rewards=[y.episode_r for y in yb if y.absolute_end]
#         print([y for y in yb if y.absolute_end])
        for r in rewards:
            if len(r.numpy().shape)!=0:self.rolling_rewards.extend(r.numpy())
            else:                      self.rolling_rewards.append(r.numpy())
#         print(len(rewards))
#         if len(rewards)!=0:self.r=sum(rewards)/len(rewards)
        
    @property
    def value(self):return np.mean(self.rolling_rewards) if len(self.rolling_rewards)!=1 else self.rolling_rewards[0]
    @property
    def name(self):return 'avg_episode_r'

In [38]:
from fastrl.actorcritic.a3c_data import *

In [39]:
env='CartPole-v1'

block=AsyncExperienceBlock(
    experience_block=partial(FirstLastExperienceBlock,a=0,seed=0,dls_kwargs={'bs':1,'n_steps':4,'num_workers':0,'verbose':False,'indexed':True,'shuffle_train':False}),
    n_processes=4,
    n=128
)
blk=IterableDataBlock(blocks=(block),
                      splitter=FuncSplitter(lambda x:False),
                      batch_tfms=lambda x:(x['s'],x),
                     )
dls=blk.dataloaders([env]*15,bs=128)

model=LinearA2C((4,),2)
agent=ActorCriticAgent(model=model)
learner=A3CLearner(dls,agent=agent,cbs=[A3CTrainer],reward_steps=4,
                   metrics=[AvgEpisodeRewardMetric()])
learner.fit(3)

epoch,train_loss,train_avg_episode_r,valid_loss,valid_avg_episode_r,time
0,1.674018,8.307692,None,8.307692,00:00
1,1.630438,8.653846,None,8.653846,00:00
2,1.587409,8.769231,None,8.769231,00:00


/opt/project/fastrl/fastrl/actorcritic/a3c_data.py:97: UserWarning: Using a target size (torch.Size([128, 1])) that is different to the input size (torch.Size([128])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss_value_v=F.mse_loss(value_v.squeeze(-1),r_est)
/opt/conda/envs/fastrl/lib/python3.7/site-packages/fastprogress/fastprogress.py:74: UserWarning: Your generator is empty.
  warn("Your generator is empty.")


In [40]:
# hide
from nbdev.export2html import *
from nbdev.export import *
notebook2script()
notebook2html(n_workers=0)

Converted 00_core.ipynb.
Converted 01_wrappers.ipynb.
Converted 03_basic_agents.ipynb.
Converted 04_learner.ipynb.
Converted 05a_data.ipynb.
Converted 05b_async_data.ipynb.
Converted 06_basic_train.ipynb.
Converted 13_metrics.ipynb.
Converted 14_actorcritic.sac.ipynb.
Converted 15_actorcritic.a3c_data.ipynb.
Converted 16_actorcritic.a2c.ipynb.
Converted index.ipynb.


converting: /opt/project/fastrl/nbs/16_actorcritic.a2c.ipynb
converting: /opt/project/fastrl/nbs/13_metrics.ipynb
